In [ ]:
%run init.ipynb

In [ ]:
preprocessor = mz.models.DynamicClip.get_default_preprocessor()

In [ ]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

In [ ]:
preprocessor.context

In [ ]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [ ]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20
)

In [ ]:
padding_callback = mz.models.DynamicClip.get_default_padding_callback(
    fixed_length_left=10,
    fixed_length_right=100,
    pad_word_value=0,
    pad_word_mode='pre'
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [ ]:
model = mz.models.DynamicClip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'max'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_topk'] = 10
model.params['right_topk'] = 5

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

In [ ]:
model = mz.models.DynamicClip()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['clip_type'] = 'threshold'
model.params['left_length'] = 10
model.params['right_length'] = 100
model.params['hidden_dim'] = 128
model.params['kernel_size'] = 3
model.params['pool_size'] = 2
model.params['left_threshold'] = 0.08
model.params['right_threshold'] = 0.08

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()